## Лабораторная работа 1


По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

Имеются следующие входные данные:

* Таблица users x movies с рейтингами. Архив с датасетом нужно скачать с сайта GroupLens. Также, он загружен на HDFS в /labs/laba01/ml-100k. Файл u.data содержит все оценки, а файл u.item — список всех фильмов.

Результат:
    
* Выходной формат файла — json из словарей

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "Spark Dataframe app") 

In [4]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [5]:
df = spark.read.csv("/labs/laba01/ml-100k/u.data", sep="\t")
df = df.withColumnRenamed("_c0", "user_id")\
       .withColumnRenamed("_c1", "item_id")\
       .withColumnRenamed("_c2", "rating")\
       .withColumnRenamed("_c3", "timestamp")
df.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [6]:
df.registerTempTable('df')

In [7]:
query = f""" 
    SELECT rating, count(*) as count
    FROM df
    WHERE item_id=302
    GROUP BY rating
    ORDER BY rating ASC
"""

my_movie = spark.sql(query).toPandas()
my_movie

,rating,count
0,1,2
1,2,10
2,3,46
3,4,119
4,5,120


In [8]:
query = f""" 
    SELECT rating, count(*) as count 
    FROM df
    GROUP BY rating
    ORDER BY rating ASC
"""

all_movies = spark.sql(query).toPandas()
all_movies

,rating,count
0,1,6110
1,2,11370
2,3,27145
3,4,34174
4,5,21201


In [9]:
result = {}
result['hist_film'] = list(my_movie['count'])
result['hist_all'] = list(all_movies['count'])

In [10]:
import json

In [11]:
json.dumps(result)

'{"hist_film": [2, 10, 46, 119, 120], "hist_all": [6110, 11370, 27145, 34174, 21201]}'

In [12]:
with open('lab01.json', 'w', encoding='utf-8') as w:
    json.dump(result, w, ensure_ascii=False, indent=4)

In [13]:
sc.stop()
